In [7]:
import tweepy
from tweepy import OAuthHandler

consumer_key = 'KYBrGn09HcJ34AyBCM4jrU8Mh'
consumer_secret = 'KYBrGn09HcJ34AyBCM4jrU8Mh'
access_token = 'KYBrGn09HcJ34AyBCM4jrU8Mh'
access_secret = 'KYBrGn09HcJ34AyBCM4jrU8Mh'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [13]:
import json

tweets = []
for line in open('stream_apple.json', 'r'):
    tweets.append(json.loads(line))


In [49]:
with open('stream_apple.json', encoding="utf8") as f:
    synopses = f.read().split('created_at')

In [50]:
synopses[:10]

['{"',
 '":"Fri Jan 26 01:07:00 +0000 2018","id":956694929111175168,"id_str":"956694929111175168","text":"RT @waypointedu: Waypoint EDU is at #Bett2018\\n\\nThe augmented-reality geocaching game for educators and families.\\n\\nJust launched  worldwide\\u2026","source":"\\u003ca href=\\"http:\\/\\/bufferapp.com\\" rel=\\"nofollow\\"\\u003eBuffer\\u003c\\/a\\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3642951,"id_str":"3642951","name":"Durff","screen_name":"Durff","location":"aka Edith","url":"http:\\/\\/lisadurff.weebly.com\\/","description":"I enable people to use technology tools to connect with their students, their audience, & each other.","translator_type":"none","protected":false,"verified":false,"followers_count":2014,"friends_count":1498,"listed_count":121,"favourites_count":1314,"statuses_count":140900,"',
 '":"Fri Apr 06 18:10:04 +00

In [17]:
import numpy as np
import pandas as pd
import nltk
import re
import os
from sklearn import feature_extraction

In [18]:
tweets[:10]

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Fri Jan 26 01:07:00 +0000 2018',
  'entities': {'hashtags': [{'indices': [36, 45], 'text': 'Bett2018'}],
   'symbols': [],
   'urls': [],
   'user_mentions': [{'id': 889460534994665472,
     'id_str': '889460534994665472',
     'indices': [3, 15],
     'name': 'Waypoint EDU',
     'screen_name': 'waypointedu'}]},
  'favorite_count': 0,
  'favorited': False,
  'filter_level': 'low',
  'geo': None,
  'id': 956694929111175168,
  'id_str': '956694929111175168',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'place': None,
  'quote_count': 0,
  'reply_count': 0,
  'retweet_count': 0,
  'retweeted': False,
  'retweeted_status': {'contributors': None,
   'coordinates': None,
   'created_at': 'Thu Jan 25 11:55:46 +0000 2018',
   'display_text_range': [0, 140],
   

In [24]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [25]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [26]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [51]:
totalvocab_stemmed = []
totalvocab_tokenized = []

for i in synopses:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [52]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 161042 items in vocab_frame


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopses) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 1min 5s, sys: 55.1 ms, total: 1min 5s
Wall time: 1min 5s
(1509, 278)


In [54]:
terms = tfidf_vectorizer.get_feature_names()

In [55]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [56]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 5.44 s, sys: 27.1 ms, total: 5.47 s
Wall time: 5.57 s


In [58]:
ranks = list(range(1,101))
len(tweets)

473

In [77]:
ranks = list(range(1,101))
films = { 'title': tweets[:100], 'rank': ranks, 'synopsis': tweets[:101], 'cluster': clusters[:100]}

frame = pd.DataFrame(films, index = [clusters[:100]] , columns = ['rank', 'title', 'cluster'])

In [78]:
frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)

4    35
3    22
2    18
0    15
1    10
Name: cluster, dtype: int64

In [79]:
grouped = frame['rank'].groupby(frame['cluster']) #groupby cluster for aggregation purposes

grouped.mean() #average rank (1 to 100) per cluster

cluster
0    56.000000
1    50.200000
2    47.333333
3    52.090909
4    48.857143
Name: rank, dtype: float64

In [80]:
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')

Top terms per cluster:

Cluster 0 words: b'id_str', b'id', b'id', b'\\u003etwitter', b'nofollow\\', b'iphone\\u003c\\/a\\u003e',

Cluster 0 titles: {'created_at': 'Fri Jan 26 01:07:00 +0000 2018', 'id': 956694930675597312, 'id_str': '956694930675597312', 'text': '@Monica78774610 Free Apple Watch Series 2 Giveaway!\nApple is Giving Away #10,000\nFree Shipping Worldwide 📦\nGrab one… https://t.co/68ySCYErLQ', 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 948283367962955777, 'in_reply_to_user_id_str': '948283367962955777', 'in_reply_to_screen_name': 'Monica78774610', 'user': {'id': 343420047, 'id_str': '343420047', 'name': 'Pokécoins Generator', 'screen_name': 'bunekokuyoneda', 'location': 'United Kingdom', 'url': 'http://111026.p0kecoinsnow.com', 'description': 'Generate Free Pokécoins!!! \nQuick and Seamlessly Pokécoins Generator within Minutes Lin

/Users/shuixin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
/Users/shuixin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


 b'indices', b'lang', b'lang', b'true', b'en', b'user_mentions',

Cluster 1 titles: {'created_at': 'Fri Jan 26 01:07:01 +0000 2018', 'id': 956694933372592128, 'id_str': '956694933372592128', 'text': 'RT @Drawentry: Enter the Line of duty #charity #competition to #win the new #iPad Pro\n\n#Free to enter click https://t.co/4VU4GirEJW to comp…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 312004375, 'id_str': '312004375', 'name': 'stephen farrow', 'screen_name': 'ste_mufc1989', 'location': 'England', 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 8, 'friends_count': 173, 'listed_count': 0, 'favourites_count': 28, 'statuses_count': 93, 'created_at': 'Mon Jun 06 12:41:01 +00